# Sentiment Analysis Inference APP with GRADIO
Sentiment Analysis is the process of ‘computationally’ determining whether a piece of writing is positive, negative or neutral. It’s also known as opinion mining, deriving the opinion or attitude of a speaker.

<img src= "https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/sentiment.jpg" width=1000>

In [1]:
!pip install onnxruntime -q
!pip install gradio==4.44.0  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 332.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58

In [2]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/tokenizer_sentiment.pkl
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/sentiment-int8.onnx

--2024-09-30 10:10:59--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/tokenizer_sentiment.pkl
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 52.95.143.98, 52.95.150.14, 3.5.245.26, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|52.95.143.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 468214 (457K) [application/octet-stream]
Saving to: ‘tokenizer_sentiment.pkl’

tokenizer_sentiment 100%[===================>] 457.24K   843KB/s    in 0.5s    

2024-09-30 10:11:00 (843 KB/s) - ‘tokenizer_sentiment.pkl’ saved [468214/468214]

--2024-09-30 10:11:00--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/sentiment-int8.onnx
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 52.95.143.98, 52.95.150.14, 3.5.245.26, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|52.95.143.98|:443... connecte

In [3]:
### MAIN SCRIPT OF INFERENCE  #######
# ONLY 3 LIB. REQUIRES

import onnxruntime
import joblib
import numpy as np

tokenizer = joblib.load("tokenizer_sentiment.pkl")  # load tokenizer
onnx_model_path = "sentiment-int8.onnx"            # load model quantized int8
ort_session = onnxruntime.InferenceSession(onnx_model_path)

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=-1, keepdims=True)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")

    # Prepare input data for ONNX model
    input_data = {
                "input_ids": inputs["input_ids"].numpy(),
                "attention_mask": inputs["attention_mask"].numpy()
                }
    output = ort_session.run(None, input_data)
    logits = np.array(output[0])
    probabilities = softmax(logits)
    sentiment_label = np.argmax(probabilities).item()
    sentiment_prob = np.max(probabilities).item()
    # Map sentiment labels to "Negative" and "Positive"
    if sentiment_label == 0:
        sentiment_str = "Negative"
    else:
        sentiment_str = "Positive"
    return sentiment_str, sentiment_prob

In [4]:
analyze_sentiment('i love my dog')

('Positive', 0.8573682308197021)

In [5]:
## UI OF INFERENCE INSIDE COLAB

import gradio as gr

title = "Sentiment Analysis Application"
description = "This application analyzes the sentiment of the given text with DistillBert Quantized (8-bit)."

demo = gr.Interface(
                    fn=analyze_sentiment,
                    inputs=gr.Textbox(placeholder="Write your text here...", label="Input Text"),
                    outputs=[gr.Textbox(label="Sentiment"), gr.Textbox(label="Sentiment Probability")],
                    title=title,
                    description=description,
                    allow_flagging= "never",
                    )

demo.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b86d3c91fedeaeb809.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b86d3c91fedeaeb809.gradio.live
